# Generate dataset for image segmentation from a live video camera

We now use a simple GUI to generate new dataset for image segmentation. 

Here are the steps:


* Get an image from a camera (could later be a video or images). 
* Click on the video to extract one image, and then click again to label the oject on the image.
* Save image, coordinate and mask to the dataset.
* Split images and mask as a train and validation dataset.

You want at least 150 imgaes to get ok tracking. We can further improve the model later on.

In [34]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import ipywidgets as widgets
from ipyevents import Event 
import threading
from IPython.display import display

from unetTracker.trackingProject import TrackingProject
from unetTracker.multiClassUNetDataset import MultiClassUNetDataset
from unetTracker.camera import USBCamera, bgr8_to_jpeg
from unetTracker.unetGUI import LabelFromCameraGUI



## Load a TrackingProject object from file



In [4]:
project = TrackingProject(name="faceTrack",root_folder = "/home/kevin/Documents/trackingProjects/")

Project directory: /home/kevin/Documents/trackingProjects/faceTrack
Loading /home/kevin/Documents/trackingProjects/faceTrack/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_size': [480, 640], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'faceTrack', 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (128.0, 0.0, 128.0)], 'objects': ['nose', 'chin', 'rEye', 'lEye'], 'target_radius': 10}


## Dataset object

A Dataset object is created to save labeled images, masks and coordinates. It is also used to load the data into memory when training your model. This is a class inherited from `torch.utils.data.Dataset`.


In [21]:
dataset = MultiClassUNetDataset(image_dir=project.image_dir, mask_dir=project.mask_dir, coordinates_dir=project.coordinates_dir)

You can know how many images are in your dataset.

In [23]:
len(dataset)

470

You can also load any of the images in your dataset, together with mask and coordinates.

In [27]:
image,mask,coordinates = dataset[0]

In this case we are getting an image, mask, and coordinates of objects.

The image and mask are `torch.tensor` instead of `np.array`

## Camera class


In [28]:
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 1 Nov 25 09:04 /dev/video1
crw-rw----+ 1 root video 81, 0 Nov 25 09:04 /dev/video0


In [29]:
camera = USBCamera(width=project.image_size[1], height=project.image_size[0])

[ WARN:0@407.447] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


RuntimeError: Could not initialize camera.  Please see error trace.

In [30]:
image = camera.read()
print(image.shape)


(480, 640, 3)


In [31]:
imgWidget = widgets.Image(format='jpeg')
imgWidget.value = bgr8_to_jpeg(image)
display(imgWidget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Start the GUI to extract and label frames from a camera 

1. Click on the top left image to capture a frame from the camera feed. A larger image will appear below the entry boxes for the coordinates. 
2. Your body parts are listed above the large image. In the large image, click on the object that is selected in the radio button. You should see a label apper in the picture at the very bottom.
3. Repeat for all the body parts visible on the image. If you leave the coordinate of an object at 0,0, it is considered not in the image.
4. Once all the body parts are labeled, click on `Save labelled frame`.
5. Go back to step 1 and repeat.

In [35]:
gui = LabelFromCameraGUI(camera,project,dataset)

In [36]:
gui.stop_video()

In [37]:
print(len(dataset))


480


When you click on `Save labelled frame`, a new data point is added to your dataset. This is stored in a subdirectory of your project directory called `dataset`. There are 3 directories there: images, masks and coordinates. This data will be used to train the network and assess tracking quality. 

# Create training and validation sets

The images and masks were saved in an `images` and `masks` directories. We need to create `train_images`, `train_masks`, `val_images` and `val_masks` directories.

We will use the validation folders to estimate the accuracy of our model on data that it has not seen during training.

In [27]:
project.image_dir,project.mask_dir

('/home/kevin/Documents/trackingProjects/faceTrack/dataset/images',
 '/home/kevin/Documents/trackingProjects/faceTrack/dataset/mask')

In [28]:
dataset.create_training_validation_dataset(train_images_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_images",
                                           train_masks_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_masks",
                                           train_coordinates_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_coordinates",
                                               
                                           val_images_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_images",
                                           val_masks_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_masks",
                                           val_coordinates_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_coordinates",
                                           test_size=0.15)

Length of training set: 277
Length of validation set: 49
Copying files to training and validation directories


In [29]:
train_images_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_images"
train_masks_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_masks"
train_coordinates_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/train_coordinates"
trainDataset = MultiClassUNetDataset(image_dir=train_images_dir, mask_dir=train_masks_dir,coordinates_dir=train_coordinates_dir)
len(trainDataset)

277

In [30]:
val_images_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_images"
val_masks_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_masks"
val_coordinates_dir = "/home/kevin/Documents/trackingProjects/faceTrack/dataset/val_coordinates"
valDataset = MultiClassUNetDataset(image_dir=val_images_dir, mask_dir=val_masks_dir,coordinates_dir=val_coordinates_dir)
len(valDataset)

49

We are now ready to train a network with this data.